# This code snippet is a Python script for real-time multi-object tracking using YOLOv5 for object detection and a Simple Online and Realtime Tracking (SORT) algorithm for tracking. Here's a breakdown of what the code does:


In [ ]:
#%pip install opencv-python
#%pip install torch
#%pip install pandas
#%pip install requests
#%pip install Pillow
#%pip install filterpy

import pandas as pd
import cv2
import numpy as np
import time
from filterpy.kalman import KalmanFilter
import time
from collections import defaultdict
import torch

# Load the YOLOv5 model (YOLOv5m is used for better accuracy)
model = torch.hub.load('ultralytics/yolov5', 'yolov5m')

# COCO class labels with custom names
class_labels = {
    2: 'Car',
    3: 'Motorbike',
    5: 'Bus',
    7: 'Truck',
    0: 'Person',
    1: 'Bicycle',
    9: 'Three-wheeler',  # Placeholder for custom class (if available)
}

# Dictionary to track object appearance times and first seen timestamps
track_times = {}
active_tracks = {}

# Capture video from webcam
cap = cv2.VideoCapture(0)

# Function to log data to a text file
def log_to_file(object_id, label, duration, first_seen, last_seen):
    with open("tracking_log.txt", "a") as file:
        file.write(f"ID: {object_id}, Type: {label}, Duration: {duration:.2f} seconds, First seen: {first_seen}, Last seen: {last_seen}\n")

# SORT Tracker Class (Simple implementation for tracking)
class SORT:
    def __init__(self):
        self.trackers = []
        self.next_id = 0

    def update(self, detections):
        updated_tracks = []
        for det in detections:
            tracker = KalmanFilter(dim_x=7, dim_z=4)
            tracker.x[:4] = np.array([det[0], det[1], det[2], det[3]]).reshape((4, 1))
            tracker.F = np.eye(7)
            tracker.H = np.eye(4, 7)
            tracker.P *= 10.
            tracker.id = self.next_id
            self.next_id += 1
            updated_tracks.append(tracker)
        return updated_tracks

# Initialize SORT tracker
tracker = SORT()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame horizontally to correct axis
    frame = cv2.flip(frame, 1)

    # Object detection using YOLOv5
    results = model(frame)
    detections = results.xyxy[0].cpu().numpy()  # Bounding boxes (x1, y1, x2, y2, confidence, class)

    # Filter vehicle and person detections
    vehicle_classes = [2, 3, 5, 7, 1, 9]  # Car, Motorbike, Bus, Truck, Bicycle, Three-wheeler
    vehicles = [d for d in detections if int(d[5]) in vehicle_classes and d[4] > 0.4]
    people = [d for d in detections if int(d[5]) == 0 and d[4] > 0.4]  # Detect people (class 0)

    # Combine vehicles and people detections
    detections = vehicles + people

    # Track current frame's object IDs
    current_frame_ids = set()

    for det in detections:
        x1, y1, x2, y2, conf, cls = det
        label = class_labels.get(int(cls), 'Unknown')
        object_id = int(x1) + int(y1)  # Simple way to generate ID (replace with better tracking logic)

        current_frame_ids.add(object_id)

        # If the object is new, record the appearance time
        if object_id not in active_tracks:
            active_tracks[object_id] = {'first_seen': time.time(), 'label': label}

        # Draw bounding box in green
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        
        # Display ID and label in red
        cv2.putText(frame, f'{label} ID: {object_id}', (int(x1), int(y1) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    # Check for objects that have left the frame
    for object_id in list(active_tracks.keys()):
        if object_id not in current_frame_ids:
            first_seen = active_tracks[object_id]['first_seen']
            last_seen = time.time()
            duration = last_seen - first_seen
            log_to_file(object_id, active_tracks[object_id]['label'], duration, first_seen, last_seen)  # Log data to file
            del active_tracks[object_id]  # Remove the object from active tracks

    # Display the frame with tracking
    cv2.imshow('Real-Time Multi-Object Tracking', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit
        break

# Release resources
cap.release()
cv2.destroyAllWindows()


# Now we included a phase where it records the objects log With the GUI and made it more accruate by yolov5m

In [1]:
# Import necessary libraries
# tkinter: For creating the GUI
# ttk: For creating styled widgets in Tkinter
# PIL: For handling images and converting between formats in Tkinter
# cv2: For capturing video and performing computer vision tasks
# threading: For running the video processing in a separate thread to keep the GUI responsive
# torch: For using the YOLOv5 model for object detection
# time: For tracking object detection times and managing state

# Load the YOLOv5 model (medium version) for object detection

# Dictionary mapping COCO class IDs to labels (specifically customized for vehicles and people)

# Class definition for TrackingApp: 
# Initializes the GUI, video capture, and manages the real-time tracking of objects using YOLOv5
# Contains methods for starting and stopping the tracking process, logging data, and processing video frames
# Displays detected objects, labels, and their respective bounding boxes on the live video feed
# Uses threading to process the video in the background without blocking the GUI

# Start and stop tracking buttons to begin and end the object detection process
# The log text area shows the logs of detected objects and their IDs in real-time

# video processing function:
# Captures each frame from the webcam, processes it for object detection, and displays the results
# Uses the YOLOv5 model to detect objects in the video frame
# Each detected object is assigned an ID and logged in the GUI
# Detected objects are highlighted with bounding boxes and labels
# Updates the displayed video feed in the GUI with each processed frame

In [ ]:
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import cv2
import threading
import torch
import time

# Load the YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5m')

# Dictionary for COCO class labels (customized for vehicles and people)
class_labels = {
    0: 'Person',
    2: 'Car',
    3: 'Motorbike',
    5: 'Bus',
    7: 'Truck',
    1: 'Bicycle',
    9: 'Three-wheeler',  # Custom class (if defined)
}

class TrackingApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Real-Time Multi-Object Tracking")
        self.root.geometry("900x700")
        self.cap = cv2.VideoCapture(0)  # Initialize webcam
        self.is_running = False
        
        # Create GUI components
        self.video_frame = ttk.Label(root)
        self.video_frame.pack(pady=20)
        
        self.start_button = ttk.Button(root, text="Start Tracking", command=self.start_tracking)
        self.start_button.pack(side=tk.LEFT, padx=20)
        
        self.stop_button = ttk.Button(root, text="Stop Tracking", command=self.stop_tracking)
        self.stop_button.pack(side=tk.RIGHT, padx=20)
        
        # Display area for logs
        self.log_text = tk.Text(root, height=10, width=100)
        self.log_text.pack(pady=10)
        
        self.active_tracks = {}

    def start_tracking(self):
        if not self.is_running:
            self.is_running = True
            self.thread = threading.Thread(target=self.process_video)
            self.thread.start()

    def stop_tracking(self):
        self.is_running = False
        self.cap.release()
        cv2.destroyAllWindows()

    def log_data(self, message):
        self.log_text.insert(tk.END, message + '\n')
        self.log_text.see(tk.END)

    def process_video(self):
        while self.is_running:
            ret, frame = self.cap.read()
            if not ret:
                break
            
            # Flip the frame horizontally
            frame = cv2.flip(frame, 1)

            # Object detection using YOLOv5
            results = model(frame)
            detections = results.xyxy[0].cpu().numpy()
            
            current_frame_ids = set()
            for det in detections:
                x1, y1, x2, y2, conf, cls = det
                if conf > 0.4:  # Confidence threshold
                    label = class_labels.get(int(cls), 'Unknown')
                    object_id = int(x1 + y1)  # Simplified ID generation
                    
                    # Track appearance times
                    current_frame_ids.add(object_id)
                    if object_id not in self.active_tracks:
                        self.active_tracks[object_id] = time.time()
                        self.log_data(f"Object {label} (ID {object_id}) detected.")
                    
                    # Draw bounding box
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                    cv2.putText(frame, f'{label} ID: {object_id}', (int(x1), int(y1) - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
            
            # Convert OpenCV frame (BGR) to Tkinter-compatible format (RGB)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(frame)
            img_tk = ImageTk.PhotoImage(image=img)
            self.video_frame.img_tk = img_tk
            self.video_frame.config(image=img_tk)
            self.root.update_idletasks()

# Run the GUI application
root = tk.Tk()
app = TrackingApp(root)
root.mainloop()


# Real-Time Multi-Object Tracking System with GUI
Overview
This project integrates a real-time multi-object tracking system using YOLOv5 with a Tkinter GUI. It captures live video from the webcam, performs object detection and tracking, and displays the results along with logs in a graphical interface.

## Key Features
Start and Stop Buttons: Control the tracking process.
Live Video Display: Shows real-time video with bounding boxes around detected objects.
Logging Section: Provides real-time logs of detected objects and their appearance times.
Dependencies
Ensure you have the following Python libraries installed:

pip install torch torchvision opencv-python pillow tkinter
Code Explanation

## Imports and Setup
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import cv2
import threading
import torch
import time
Tkinter: Provides GUI components.
Pillow (PIL): Handles image conversion for display in Tkinter.
OpenCV: Captures video from the webcam and processes frames.
PyTorch: Loads and runs the YOLOv5 model.
Threading: Allows video processing to run concurrently with the GUI.

## Load YOLOv5 Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5m')
YOLOv5m: Medium-sized YOLOv5 model for balanced performance and speed.

## GUI Class Definition
class TrackingApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Real-Time Multi-Object Tracking")
        self.root.geometry("900x700")
        ...
TrackingApp class: Defines the main GUI and functionality.
Window Setup: Sets title and size.
Video Source: Captures video from the default webcam.

## GUI Components
self.video_frame = ttk.Label(root)
self.video_frame.pack(pady=20)

self.start_button = ttk.Button(root, text="Start Tracking", command=self.start_tracking)
self.start_button.pack(side=tk.LEFT, padx=20)

self.stop_button = ttk.Button(root, text="Stop Tracking", command=self.stop_tracking)
self.stop_button.pack(side=tk.RIGHT, padx=20)

self.log_text = tk.Text(root, height=10, width=100)
self.log_text.pack(pady=10)
video_frame: Displays the live video feed.
start_button and stop_button: Control the tracking process.
log_text: Logs detected objects and their IDs.

## Start and Stop Functions
def start_tracking(self):
    if not self.is_running:
        self.is_running = True
        self.thread = threading.Thread(target=self.process_video)
        self.thread.start()

def stop_tracking(self):
    self.is_running = False
    self.cap.release()
    cv2.destroyAllWindows()
start_tracking: Initiates a separate thread for video processing.
stop_tracking: Stops the tracking process and releases resources.

## Video Processing and Detection
def process_video(self):
    while self.is_running:
        ret, frame = self.cap.read()
        if not ret:
            break
        
        results = model(frame)  # Object detection using YOLOv5
        detections = results.xyxy[0].cpu().numpy()
        ...
        
        # Convert and display the frame
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        img_tk = ImageTk.PhotoImage(image=img)
        self.video_frame.img_tk = img_tk
        self.video_frame.config(image=img_tk)
        self.root.update_idletasks()
Frame Capture: Reads frames from the webcam.
Object Detection: Runs YOLOv5 on each frame and retrieves detection results.
Frame Display: Converts the frame to RGB and displays it in the GUI.

## Logging Mechanism
def log_data(self, message):
    self.log_text.insert(tk.END, message + '\n')
    self.log_text.see(tk.END)
Logs detected objects and their IDs in the Text widget.
Customization Options
Class Labels: Modify the class_labels dictionary to track different object classes.
Confidence Threshold: Adjust the conf > 0.4 threshold for detection sensitivity.
Video Source: Change self.cap = cv2.VideoCapture(0) to use a different video source or file.
Running the Application
Ensure your webcam is connected.

## Run the script:
python tracking_app.py
Use the Start and Stop buttons to control tracking.